# Intro 

```
# run anaconda prompt as admin

# Using conda create a new environment 
# newenv can be whatever name you like 

    conda create --name newenv  #  


# activate the new environment
    conda activate newenv

# Now go to https://pytorch.org/get-started/locally/
# select your system config 
# copy the provided command 
# my example 
    conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch
    
# cross your fingers

# If you want to access from jupyter
    conda install ipykernel   
    
# Some users my need this as well
    conda install -c anaconda ipython_genutils
    
# You'll know if you need this because you get some errors about ipython_genutils not found
    
```
Once you've completed the above try the following cell

In [2]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))


1.10.0
True
0
1
NVIDIA GeForce RTX 2070


## Next steps

We will be going through the tutorial found here  
    https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
    

**Problems?**
Google is your friend 

Do not go any further down without the above cell working!!

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [5]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [6]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [8]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Model Parametrization

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# In a single training loop, the model makes predictions 
#    on the training dataset (fed to it in batches), and 
#    backpropagates the prediction error to adjust the model’s parameters.

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.291656  [    0/60000]
loss: 2.282990  [ 6400/60000]
loss: 2.262465  [12800/60000]
loss: 2.266156  [19200/60000]
loss: 2.243651  [25600/60000]
loss: 2.216843  [32000/60000]
loss: 2.227804  [38400/60000]
loss: 2.190175  [44800/60000]
loss: 2.187059  [51200/60000]
loss: 2.165934  [57600/60000]
Test Error: 
 Accuracy: 45.8%, Avg loss: 2.154778 

Epoch 2
-------------------------------
loss: 2.162262  [    0/60000]
loss: 2.150027  [ 6400/60000]
loss: 2.091354  [12800/60000]
loss: 2.116227  [19200/60000]
loss: 2.065780  [25600/60000]
loss: 2.012895  [32000/60000]
loss: 2.037944  [38400/60000]
loss: 1.956496  [44800/60000]
loss: 1.960935  [51200/60000]
loss: 1.903061  [57600/60000]
Test Error: 
 Accuracy: 55.5%, Avg loss: 1.888778 

Epoch 3
-------------------------------
loss: 1.918312  [    0/60000]
loss: 1.880994  [ 6400/60000]
loss: 1.762298  [12800/60000]
loss: 1.817038  [19200/60000]
loss: 1.711533  [25600/60000]
loss: 1.667223  [32000/600

## Saving a Model

In [13]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Reload a saved model

In [14]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

## Rerun a saved model

In [15]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
